
# Planet Features API Introduction using Python

---

## Introduction

---

This tutorial is an introduction to [Planet](https://www.planet.com)'s [Features API.](https://api.planet.com/features/v0/ogc/my)

The focus of this tutorial will be the creating and using AOI references within other Planet APIs. We will create a `FeatureCollection` and then create two `Features` and use them in a few other Planet APIs. After completing this tutorial, you should feel comfortable interacting with the Features API, and have a good foundation for leveraging them across the Planet API ecosystem.


## Table of Contents

---

* **Introduction**
    * Requirements
       * Software & Modules
       * Planet API Key  
    * Useful Links
    

* **Overview**
     * [The Basic Setup](#Basic-Setup)
     * [First Feature Collection](#Let's-Create-Our-First-Feature-Collection)
     * [Adding Features](#Adding-Features-to-our-Collection)
     * [Validating Features](#Validating-Features-before-Create)
     * [Getting Alternate Features](#Get-an-alternative)
     * [Using References](#Using-the-references)
       * [Data API](#Data-API)
       * [Orders API](#Orders-API)
       * [Subscriptions API](#Subscriptions-API)

### Requirements

---

#### Software & Modules

This tutorial assumes familiarity with the [Python](https://python.org) programming language throughout. Familiarity with basic REST API concepts and usage is also assumed.

We'll be using a **"Jupyter Notebook"** (aka Python Notebook) to run through the examples.
To learn more about and get started with using Jupyter, visit: [Jupyter](https://jupyter.org/) and [IPython](https://ipython.org/). 

For the best experience, download this notebook and run it on your system, and make sure to install the modules listed below first. You can also copy the examples' code to a separate Python files an run them directly with Python on your system if you prefer.


#### Planet API Key

You should have an account on the Planet Platform to access the Data API. You may retrieve your API key from your [account page](https://www.planet.com/account/), or from the "API Tab" in [Planet Explorer](https://www.planet.com/explorer).

### Useful links 

---

* [Planet Features API Reference & Documentation](https://developers.planet.com/docs/apis/features)
* [Planet Explorer](https://www.planet.com/explorer)

## Overview



## Basic Setup

---

Before interacting with the Planet Features API using Python, we will set up our environment with some useful modules and helper functions.

* We'll configure *authentication* to the Planet Features API
* We'll use the `requests` Python module to make HTTP communication easier. 
* We'll use the `json` Python module to help us work with JSON responses from the API.
* We'll create a function called `p` that will print Python dictionaries nicely.


pip install -q requests pandas geopandas 


Let's start by importing needed packages configuring authentication:

### Import Packages & Define Helper Functions

In [1]:
import os
import json
import requests
import time

# Helper function to printformatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))

### Authentication

Authentication with the Planet Features API be achieved using a valid Planet **API key**.

You can *export* your API Key as an environment variable on your system:

`export PL_API_KEY="YOUR API KEY HERE"`

Or add the variable to your path, etc. If you are using our Docker environement, it will already be set.

To start our Python code, we'll setup an API Key variable from an environment variable to use with our requests:

In [2]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = 'PASTE_YOUR_API_KEY_HERE'

    # construct auth tuple for use in the requests library
BASIC_AUTH = (API_KEY, '')

### Our First Request

In [3]:
# Setup Planet Features API base URL
URL = "https://api.planet.com/features/v0/ogc/my/collections"

# Setup the session
session = requests.Session()
# Authenticate
session.auth = (API_KEY, "")

In [4]:
# Make a GET request to the Planet Features API
res = session.get(URL)
# Now we should get a response, hopefully it's a `200` code, saying everything is OK!
res.status_code

200

## Let's Create Our First Feature Collection

---

To upload an AOI Feature you must first create a Feature Collection. To create a Feature Collection make a POST request to the `/collections` endpoint providing a title and optionally a description.

The title will be used to create a "slug" that will be part of the collection's id.

A Collection identifier is a combination of optional user-supplied title and a hashid. For example, if a user creates a Collection with the title set to "My Great Places", the URL path identifier used to reference the Collection will be my-great-places-HASHID (where HASHID might be 4wr3wrz, for example). The Collection may also be referenced using only the HASHID - this way the user may change the title of their Collection if needed.


In [5]:


request = {
    "title" : "My Title",
    "description" : "A description of this collection"
}

# Send the POST request to the API stats endpoint
res = session.post(URL, json=request)

# Print response
p(res.json())

# let's save the collection-id to use later
collection_id = res.json()["id"]

{
  "id": "my-title-8vdGGZe",
  "title": "My Title",
  "description": "A description of this collection",
  "item_type": "feature",
  "links": [
    {
      "href": "https://api.planet.com/features/v0/ogc/my/collections/my-title-8vdGGZe",
      "rel": "self",
      "title": "This collection",
      "type": "application/json"
    },
    {
      "href": "https://api.planet.com/features/v0/ogc/my/collections/my-title-8vdGGZe/items",
      "rel": "features",
      "title": "Features",
      "type": "application/json"
    }
  ],
  "permissions": {
    "can_write": true,
    "shared_with_my_org": false
  }
}


***Good Job!*** You've created your first Feature Collection

## Adding Features to our Collection 
The features API guarantees that your geojson will work in Planet's Platform.

*Note* there is a 1500 vertex limit

In [6]:

# Put your own Polygon here!
geo1 = {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -125.29632568359376,
          48.37084770238366
        ],
        [
          -125.29632568359376,
          49.335861591104106
        ],
        [
          -123.2391357421875,
          49.335861591104106
        ],
        [
          -123.2391357421875,
          48.37084770238366
        ],
        [
          -125.29632568359376,
          48.37084770238366
        ]
      ]
    ]
  }


request = {
    "geometry": geo1,
    "id": "1231", #optional, if you want to give your feature it's own id, use that here. this needs to be unique
    "properties" : {
        "title": "some-title",
        "description": "some-description"
    }

}

res = session.post(URL +"/"+ collection_id + "/items", json=request)


# Let's save the response as the short ref for later!
short_ref = res.json()[0]

print(short_ref)

pl:features/my/my-title-8vdGGZe/1231-amXKR50


In [7]:
# add a second feature
geo2 = {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -124.87509770059592,
              48.42395972830914
            ],
            [
              -124.58981375958084,
              47.620415719416116
            ],
            [
              -124.36822528967289,
              47.28340570291758
            ],
            [
              -124.20764093293805,
              47.048428715791005
            ],
            [
              -122.64306485391211,
              47.097307284357186
            ],
            [
              -122.7148343988215,
              48.16372351943102
            ],
            [
              -123.31052162156989,
              48.14326012054822
            ],
            [
              -123.9995092527008,
              48.23068727937715
            ],
            [
              -124.87509770059592,
              48.42395972830914
            ]
          ]
        ]
      }


request = {
    "geometry": geo2,
    "id": "1235", #optional, if you want to give your feature it's own id, use that here. this needs to be unique
    "properties" : {
        "title": "some-title",
        "description": "some-description"
    }

}

res = session.post(URL +"/"+ collection_id + "/items", json=request)


# Let's save the response as the short ref for later!
short_ref_2 = res.json()[0]

print(short_ref_2)

pl:features/my/my-title-8vdGGZe/1235-VmkvW40


## Getting/Editing Features

Once you've created features you can list all, or just get one. You can also edit a feature after you've created it, it will create a new version of that feature. 
View [the documentation to learn more](https://developers.planet.com/docs/apis/features/upload-and-validate/#accessing-your-features)

## Validating Features before Create

---

The Features API enables you to check if your Feature geometry is valid for use in Planet's Platform before uploading it to your Feature Collection. The `/validate` endpoint provides more details as to why a geometry is invalid, such as not meeting the vertex limit. The validation endpoint only supports validating Features.  
*i.e., you cannot validate a Feature Collection.*



In [8]:
validate_url =  'https://api.planet.com/features/v0/ogc/my/collections/validate'

# A request with no issues
request = {
    "type": "Feature", 
    "properties": {},
    "geometry": geo1,
    }


res = session.post(validate_url, json=request)
p(res.json())

{
  "status": "okay",
  "detail": "No errors."
}


### Let's try a bow tie, this request will return an error

In [9]:
# A request with problems

bow_tie = {
        "coordinates": [
            [
              [
                -110.19612631421622,
                35.143918734708365
              ],
              [
                -106.04640407969373,
                36.00744957163177
              ],
              [
                -106.28152102495591,
                33.78198439977524
              ],
              [
                -110.06298522792459,
                36.426344365077654
              ],
              [
                 -110.19612631421622,
                35.143918734708365
              ]
            ]
        ],
        "type": "Polygon"
      }

request = {
    "type": "Feature", 
    "properties": {},
    "geometry": bow_tie,
    }


res = session.post(validate_url, json=request)
p(res.json())

{
  "errors": [
    {
      "code": "invalid-geometry",
      "description": "Geometry is invalid: Self-intersection[-108.680204575141 35.4593724190932]"
    }
  ],
  "description": "To see valid version of the geometry, post to the alternates_url",
  "alternates_url": "https://api.planet.com/features/v0/ogc/my/collections/alternates"
}


**Note:** When you get an invalid geojson, we will try to offer alternate solutions, checkout the `/alternates` endpoint.

### Let's try getting alternates for a feature with more than 1500 vertices

In [10]:
# read this from a file
complex_geom_file = os.path.join('warren-county.geojson')

def load_geojson(filename):
    with open(filename, 'r') as f:
        return json.load(f)

complex_geom = load_geojson(complex_geom_file)

In [11]:
request = complex_geom


res = session.post(validate_url, json=request)
p(res.json())

{
  "errors": [
    {
      "code": "too-many-coordinates",
      "description": "Geometry has more than 1500 points."
    }
  ],
  "description": "To see valid version of the geometry, post to the alternates_url",
  "alternates_url": "https://api.planet.com/features/v0/ogc/my/collections/alternates"
}


### Get an alternative

In [12]:
alternate_url = 'https://api.planet.com/features/v0/ogc/my/collections/alternates'
request = complex_geom



res = session.post(alternate_url, json=request)
alternates = res.json()

# simplify the print here, we don't need to output all geometry points (cause it's a lot!)
for item in alternates["features"]:
    del item["geometry"]



p(alternates)

# if you want to see the whole output, comment the line below back in
# p(res.json())

{
  "type": "FeatureCollection",
  "features": [
    {
      "properties": {
        "technique": "Bounding box"
      }
    },
    {
      "properties": {
        "technique": "Convex hull"
      }
    },
    {
      "properties": {
        "technique": "Simplified to ~10m"
      }
    }
  ]
}


*Note:* This endpoint provides provides recommendations and details about the technique used to alter the provided geometry so that it meets Planet's validity requirements.
Pick which geometry makes sense for you.

## Using the references 

Features references are supported in the Data API, Subscriptions API and Orders API.
We'll walk through some examples together below.

For all APIs you can replace your current geometry geojson block requests with a block like this:
```json
  "geometry": {
    "content": "pl:features/my/[collection]/[feature-id]", # your short ref
    "type": "ref" # instead of a Polygon or Multipolygon, the type is `ref`
  },

```

### Data API
#### Quick Searches with a Feature Reference

In [13]:
# Setup Item Types
item_types = ["PSScene"]


# Setup a geometry filter
geometry = {
    "type": "ref",
    "content": short_ref,
}

# Let's filter for the month of jan, and images that are not too cloudy.
filter = {
   "type":"AndFilter",
   "config":[
        {
            "type":"DateRangeFilter",
            "field_name":"acquired",
            "config":{
               "gte":"2024-01-01T00:00:00Z",
               "lte":"2024-01-15T00:00:00Z"
            },
        }, 
        {
           "type":"RangeFilter",
           "field_name":"cloud_cover",
           "config":{
              "lte":0.1
           },
        }
   ]
}


# Setup the request
request = {
    "item_types" : item_types,
    "geometry" : geometry,
    "filter": filter
    
}

In [ ]:
# Send the POST request to the API quick search endpoint
quick_url =  "https://api.planet.com/data/v1/quick-search"
res = session.post(quick_url, json=request)


# Leaving the response commented out as it's quite long
# Print the response
# p(res.json())



Nice! You'll notice the geometry that was used to seach becomes hydrated in the response.


In [15]:
# save an item ID for the orders request below.
item_id = res.json()["features"][0]["id"]


*Note* The geometry field in the Data API can also take an array of refs

In [16]:
# Setup the request
multiple_ref_geometry = {
    "type": "ref",
    "content": [short_ref, short_ref_2],
}

request = {
    "item_types" : item_types,
    "geometry" : geometry,
    "filter": filter
    
}

In [17]:
# Send the POST request to the API quick search endpoint
res = session.post(quick_url, json=request)


# Leaving the response commented out as it's quite long
# Print the response
# p(res.json())

#### Saved Searches with a Feature Reference

---

The `/searches` endpoint lets you created saved searches that can be reused.

To view your saved searches, visit the [`searches/?search_type=saved`](https://api.planet.com/data/v1/searches/?search_type=saved) endpoint.

Finally, let's create a saved search using a feature reference:

In [18]:
# Setup the saved searches endpoint url
searches_url = "https://api.planet.com/data/v1/searches"

In [19]:
# Setup the request, specifying a name for the saved search, along with the usual search item_types and filter.

request = {
    "name" : "My Feature reference search",
    "item_types" : item_types,
    "filter" : filter,
    "geometry": geometry
}

# Send a POST request to the saved searches endpoint (Create the saved search)
res = session.post(searches_url, json=request)

# Print the response
p(res.json())

{
  "__daily_email_enabled": false,
  "_links": {
    "_self": "https://api.planet.com/data/v1/searches/b74387368b804debad6b2c5d3c2edebd",
    "results": "https://api.planet.com/data/v1/searches/b74387368b804debad6b2c5d3c2edebd/results"
  },
  "created": "2024-01-30T22:48:18.634632Z",
  "filter": {
    "config": [
      {
        "config": {
          "gte": "2024-01-01T00:00:00Z",
          "lte": "2024-01-15T00:00:00Z"
        },
        "field_name": "acquired",
        "type": "DateRangeFilter"
      },
      {
        "config": {
          "lte": 0.1
        },
        "field_name": "cloud_cover",
        "type": "RangeFilter"
      }
    ],
    "type": "AndFilter"
  },
  "geometry": {
    "content": [
      "pl:features/my/my-title-8vdGGZe/1231-amXKR50#1"
    ],
    "type": "ref"
  },
  "id": "b74387368b804debad6b2c5d3c2edebd",
  "item_types": [
    "PSScene"
  ],
  "last_executed": null,
  "name": "My Feature reference search",
  "search_type": "saved",
  "updated": "2024-01-30T

*Note* you can see the reference returned in the search results

Now that we created a saved search, let's get a list of our saved searches:

In [20]:
# Send a GET request to the saved searches endpoint with the saved search type parameter (Get saved searches)
res = session.get(searches_url, params={"search_type" : "saved"})

p(res.json())


{
  "_links": {
    "_first": "https://api.planet.com/data/v1/searches/?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogImNyZWF0ZWQiLCAic29ydF9kZXNjIjogdHJ1ZSwgInNvcnRfc3RhcnQiOiBudWxsLCAic29ydF9sYXN0X2lkIjogbnVsbCwgInNvcnRfcHJldiI6IGZhbHNlLCAicXVlcnlfcGFyYW1zIjogeyJzZWFyY2hfdHlwZSI6ICJzYXZlZCJ9fQ%3D%3D",
    "_next": "https://api.planet.com/data/v1/searches/?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogImNyZWF0ZWQiLCAic29ydF9kZXNjIjogdHJ1ZSwgInNvcnRfc3RhcnQiOiAiMjAyNC0wMS0yM1QyMjoxODozMC4xNTY2MzhaIiwgInNvcnRfbGFzdF9pZCI6ICIyNjYyMjA3OTU4IiwgInNvcnRfcHJldiI6IGZhbHNlLCAicXVlcnlfcGFyYW1zIjogeyJzZWFyY2hfdHlwZSI6ICJzYXZlZCJ9fQ%3D%3D",
    "_prev": "https://api.planet.com/data/v1/searches/?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogImNyZWF0ZWQiLCAic29ydF9kZXNjIjogdHJ1ZSwgInNvcnRfc3RhcnQiOiAiMjAyNC0wMS0zMFQyMjo0ODoxOC42MzQ2MzJaIiwgInNvcnRfbGFzdF9pZCI6ICIyNjY5NjI0ODg5IiwgInNvcnRfcHJldiI6IHRydWUsICJxdWVyeV9wYXJhbXMiOiB7InNlYXJjaF90eXBlIjogInNhdmVkIn19",
    "_self": "https://api.planet.com/da

And you'll see the reference returned when you query your saved searches.

### Orders API

In [21]:
orders_url = "https://api.planet.com/compute/ops/orders/v2"

In [22]:
request = {
    "name": "Test order with a reference",
    "source_type": "scenes",
    "products": [
        {
            "item_ids": [
                item_id # the id from the search request
            ],
            "item_type": "PSScene",
            "product_bundle": "analytic_udm2"
        }
    ],
    "tools": [
        {
            "clip": {
                "aoi": {
                    "content": short_ref,
                    "type": "ref"
                }
            }
        }
    ]

}

# Send a POST request to the orders endpoint
# WARNING: this will use your quota!
res = session.post(orders_url, json=request)

# Print the response
p(res.json())
# make the request

{
  "_links": {
    "_self": "https://api.planet.com/compute/ops/orders/v2/e21d91b3-c860-45c7-922a-e452fca9d4d9"
  },
  "created_on": "2024-01-30T22:48:19.406Z",
  "error_hints": [],
  "id": "e21d91b3-c860-45c7-922a-e452fca9d4d9",
  "last_message": "Preparing order",
  "last_modified": "2024-01-30T22:48:19.406Z",
  "name": "Test order with a reference",
  "products": [
    {
      "item_ids": [
        "20240114_191157_82_2488"
      ],
      "item_type": "PSScene",
      "product_bundle": "analytic_udm2"
    }
  ],
  "source_type": "scenes",
  "state": "queued",
  "tools": [
    {
      "clip": {
        "aoi": {
          "content": "pl:features/my/my-title-8vdGGZe/1231-amXKR50",
          "type": "ref"
        }
      }
    }
  ]
}


**Note** Checkout that AOI reference in the response!!

### Subscriptions API

In [23]:
# set up the subscriptions url
subs_url = "https://api.planet.com/subscriptions/v1"
    
# set up your bucket delivery
delivery_type = "your-delivery-type"
delivery_bucket = "your-bucket-name"
delivery_creds = "your-creds"

**Note** You can use the short_ref in the source geometry and optionally in the clip geometry 
If you choose to clip your order with a Features refence, it must be the same reference used in the source geometry.

In [24]:
# set up the subscription request

request = {
    "name": "Example Subscription",
    "source": {
        "type": "catalog",
        "parameters": {
            "geometry": {
                "content": short_ref,
                "type": "ref"
            },
            "time_range_type": "acquired",
            "start_time": "2024-01-01T00:00:00Z",
            "end_time": "2024-01-15T00:00:00Z",
            "item_types": [
                "PSScene"
            ],
            "asset_types": [
                "ortho_analytic_4b"
            ]
        }
    },
    "tools": [{
        "type": "clip",
        "parameters": {
            "aoi": {
                "content": short_ref,
                "type": "ref"
            }
        }
    }],
    "delivery": {
        "type": delivery_type,
        "parameters": {
            "bucket": delivery_bucket,
            "credentials": delivery_creds
        }
    }
}

# Send a POST request to the subscription endpoint
# WARNING: this will use your quota!
res = session.post(subs_url, json=request)

# Print the response
p(res.json())

{
  "name": "Example Subscription",
  "source": {
    "type": "catalog",
    "parameters": {
      "asset_types": [
        "ortho_analytic_4b"
      ],
      "item_types": [
        "PSScene"
      ],
      "geometry": {
        "coordinates": [
          [
            [
              -125.29632568359376,
              48.37084770238366
            ],
            [
              -125.29632568359376,
              49.335861591104106
            ],
            [
              -123.2391357421875,
              49.335861591104106
            ],
            [
              -123.2391357421875,
              48.37084770238366
            ],
            [
              -125.29632568359376,
              48.37084770238366
            ]
          ]
        ],
        "type": "Polygon"
      },
      "geom_ref": "pl:features/my/8vdGGZe/amXKR50#1",
      "start_time": "2024-01-01T00:00:00Z",
      "end_time": "2024-01-15T00:00:00Z",
      "publishing_stages": [
        "standard",
        "finali

**Note** You'll see the Features reference and the geojson returned in the subscriptions response.

## Conclusion 
**Congratulations!!**
You made it! That's how you can use and reuse geometry references across the Planet Platform. Go forth to use and reuse your geometry without needing to carry your geojsons around! 